## Learn about class and pytorch
runs with 3.8.2 on linux and 3.8.8 on mac

In [96]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Setup tensorboard

In [97]:
#!pip install tensorflow
#!pip install tensorflow --upgrade
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

# setup wandb

In [98]:
import wandb
wandb.init(project="testPytorch")

2021-07-22 11:05:37.990910: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-22 11:05:37.990928: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Build data loader

In [99]:
import pyarrow.feather as feather
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd



In [100]:
def build_dataset(batchSizeTrain, batchsizeValid):
    fluxData_df = feather.read_feather('data/fluxData.feather')
    # normalize input data
    fluxData_df_norm = (fluxData_df - fluxData_df.mean(axis=0)) / fluxData_df.std(axis=0)
    zernikeData_df = feather.read_feather('data/zernikeData.feather')
    X_train, X_val, y_train, y_val = train_test_split(fluxData_df_norm, zernikeData_df, test_size=0.2, random_state=42)


    train_target = torch.tensor(y_train.values.astype(np.float32))
    trainInput = torch.tensor(X_train.values.astype(np.float32))

    train_tensor = torch.utils.data.TensorDataset(trainInput, train_target) 
    loaderTrain = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batchSizeTrain, shuffle = True)


    valid_target = torch.tensor(y_val.values.astype(np.float32))
    validInput = torch.tensor(X_val.values.astype(np.float32))

    valid_tensor = torch.utils.data.TensorDataset(validInput, valid_target) 
    loaderValid = torch.utils.data.DataLoader(dataset = valid_tensor, batch_size = batchsizeValid, shuffle = True)
    return loaderTrain, loaderValid



# Build data loader

## Build AO network as class

In [101]:
config = {
"Layer1" : 2000,
"Layer2" : 12




}

In [102]:
config["Layer1"] 

2000

In [103]:
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self, Layer1):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(19,Layer1)
        self.linear2 = nn.Linear(Layer1,100)
        self.linear3 = nn.Linear(1050,100)
        self.linear4 = nn.Linear(400,400)        
        self.out = nn.Linear(100,9)
        self.dropout = nn.Dropout(p=0.0001)
        self.activations = nn.ModuleDict({
            'relu': nn.ReLU(),
            'lrelu': nn.LeakyReLU()
    })

    def forward(self, x, act = "relu"):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.dropout(x)        
        x = self.linear2(x)
        x = F.relu(x)
        x = self.dropout(x)        

        # x = self.linear3(x)
        # x = F.relu(x)
        # x = self.dropout(x)        

      #  x = self.linear4(x)
      #  x = F.relu(x) 
       # x = self.dropout(x)        
        x = self.out(x)
        return(x)


In [104]:
def train(config=None):
    print(config)
    ClassNetwork = Net(config["Layer1"]).to(device)
    wandb.watch(ClassNetwork, log_freq=10)

    optimizer = optim.AdamW(ClassNetwork.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, cooldown=5, patience=5,
    verbose=True)
    loaderTrain, loaderValid =build_dataset(128,128)
    epochs = 100
    for epoch in range(epochs):
        ClassNetwork.train()
        for i, data in enumerate(loaderTrain,0):
            input, labels = data
            input = input.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
   #         outputs = ClassNetwork(input, "relu")
            loss = nn.MSELoss()
            loss =loss(ClassNetwork(input, "relu"), labels)
    # ⬅ Backward pass + weight update
            loss.backward()
            optimizer.step()
        for i, data in enumerate(loaderValid,0):
            input, labels = data
            input = input.to(device)
            labels = labels.to(device)            
            ClassNetwork.eval()
            with torch.no_grad():
                lossVal = nn.MSELoss()            
                lossVal =lossVal(ClassNetwork(input, "relu"), labels)
                RmsLossValid=torch.sqrt(lossVal) 
        
        scheduler.step(RmsLossValid.item())            
        print("epoch: ", epoch, "loss: ", "%.6f" % loss.item() ,"RmsLossValid: " , "%.6f" % RmsLossValid.item())  
        writer.add_scalar('Loss/train', loss.item(), epoch)
        writer.add_scalar('RmsLossValid/valid', RmsLossValid.item(), epoch)
        wandb.log({"loss": loss, "RmsLossValid": RmsLossValid})
    data, labels = next(iter(loaderValid))
    data = data.to(device)
    labels = labels.to(device)    
    writer.add_graph(ClassNetwork, data  )
    writer.close
    torch.save({
            'epoch': epoch,
            'model_state_dict': ClassNetwork.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            "RmsLossValid": RmsLossValid
            }, "PaperModel.pt")

In [105]:
""" ClassNetwork = Net()
print(ClassNetwork)
optimizer = optim.AdamW(ClassNetwork.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
loaderTrain, loaderValid =build_dataset(128,128)
epochs = 8
for epoch in range(epochs):
    ClassNetwork.train()
    for i, data in enumerate(loaderTrain,0):
        input, labels = data
        optimizer.zero_grad()
        outputs = ClassNetwork(input, "relu")
        loss = nn.MSELoss()
        loss =loss(ClassNetwork(input, "relu"), labels)
# ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()
    for i, data in enumerate(loaderValid,0):
        ClassNetwork.eval()
        lossVal = nn.MSELoss()            
        lossVal =lossVal(ClassNetwork(input, "relu"), labels)
        RmsLossValid=torch.sqrt(loss) 
        scheduler.step(RmsLossValid)   
    print(optimizer.param_groups[0]["lr"])
    print("epoch: ", epoch, "loss: ", "%.4f" % loss.item() ,"RmsLossValid: " , "%.4f" % RmsLossValid.item())  
 """

' ClassNetwork = Net()\nprint(ClassNetwork)\noptimizer = optim.AdamW(ClassNetwork.parameters(), lr=0.001)\nscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, \'min\')\nloaderTrain, loaderValid =build_dataset(128,128)\nepochs = 8\nfor epoch in range(epochs):\n    ClassNetwork.train()\n    for i, data in enumerate(loaderTrain,0):\n        input, labels = data\n        optimizer.zero_grad()\n        outputs = ClassNetwork(input, "relu")\n        loss = nn.MSELoss()\n        loss =loss(ClassNetwork(input, "relu"), labels)\n# ⬅ Backward pass + weight update\n        loss.backward()\n        optimizer.step()\n    for i, data in enumerate(loaderValid,0):\n        ClassNetwork.eval()\n        lossVal = nn.MSELoss()            \n        lossVal =lossVal(ClassNetwork(input, "relu"), labels)\n        RmsLossValid=torch.sqrt(loss) \n        scheduler.step(RmsLossValid)   \n    print(optimizer.param_groups[0]["lr"])\n    print("epoch: ", epoch, "loss: ", "%.4f" % loss.item() ,"RmsLossValid

In [106]:
# ClassNetwork = Net()

# loaderTrain, loaderValid =build_dataset(1024,1024)
# data, labels = next(iter(loaderValid))
# ClassNetwork( data, "relu")

In [107]:
train(config)

{'Layer1': 2000, 'Layer2': 12}
epoch:  0 loss:  0.001488 RmsLossValid:  0.045717
epoch:  1 loss:  0.001771 RmsLossValid:  0.037547
epoch:  2 loss:  0.000724 RmsLossValid:  0.034657
epoch:  3 loss:  0.000916 RmsLossValid:  0.034765
epoch:  4 loss:  0.000930 RmsLossValid:  0.032185


KeyboardInterrupt: 